In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # Visualization

# ## 00. load package 

# In[102]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import seaborn as sns
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt7
import colorsys

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
sns.set_style('whitegrid')


# ## 00. load data

# In[103]:


pd.DataFrame(sns.get_dataset_names()).T   # available datasets


# In[104]:


# penguine data: df_peng
df_peng = sns.load_dataset('penguins')
display(df_peng.shape, df_peng.head(1), df_peng.tail(1))


# ## 01. KDE plot

# #### single plot

# In[105]:


fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6,2),
                        constrained_layout=True,
                        sharex=True, sharey=True)
sns.kdeplot(x='body_mass_g', data=df_peng, ax=axs)


# #### Plots: (1 row, 2 columns)

# In[106]:


fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12,2),   # nrows, ncols, figsize 설정
                        constrained_layout=True,            # subplot들 간의 간격을 최적으로 자동 조정
                        sharex=True, sharey=True)           # x축공유, y축공유 설정

sns.kdeplot(x='bill_length_mm', data=df_peng, ax=axs[0])
sns.kdeplot(x='bill_depth_mm', data=df_peng, ax=axs[1])

print('***************************************************')
print('** Axes:', axs[0], axs[1])
print('***************************************************')

plt.show()


# ## 02. Accumulative KDE plot

# #### single plot

# In[107]:


fig, axs = plt.subplots(figsize=(6,2), constrained_layout=True, sharex=True, sharey=True)

# 'multiple' parameter
sns.kdeplot(x='body_mass_g', data=df_peng, 
            hue='sex', multiple='stack', 
            alpha=1, linewidths=0, ax=axs)

plt.show()


# #### 다양한 parameter 활용

# In[108]:


fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12,2), 
                        constrained_layout=True, 
                        sharex=True, sharey=True)

# basic stacked plot
sns.kdeplot(x='body_mass_g', data=df_peng, hue='sex', multiple='stack', 
            alpha=1, linewidths=0, ax=axs[0])

# 다양한 parameter 활용: palette, hue_order 등
sns.kdeplot(x='body_mass_g', data=df_peng, hue='sex', multiple='stack', 
            alpha=0.5, linewidths=0, ax=axs[1],
            palette=['darkred', 'C2'], hue_order=['Female', 'Male'])

plt.show()


# ## 03. Accumulative KDE plot - Details

# ##### UDF: set_hls( )
#  - 기능: ...
#  - h: dh >> change in hue (float)
#  - l: dl >> change in Lightness (float)
#  - s: ds >> change in Saturation (float)

# #### settings

# In[109]:


def set_hls(c, dh=0, dl=0, ds=0, dalpha=0):
    """
    c : (array -like, str) color in RGB space
    dh : (float) change in Hue
         default = 0
    dl : (float) change in Lightness
         default = 0
    ds : (float) change in Saturation
         default = 0
    dalpha : opacity(투명도)
         default = 0
         >> 보통 alpha 값이 1이면 완전 불투명, 0이면 완전 투명
    """

    # 입력된 color를 RGBA numpy array로 변환
    c_rgba = mcolors.to_rgba(c)
    
    # RGB와 alpha 분리
    c_rgb = c_rgba[:3]
    alpha = c_rgba[3]
    
    # RGB 색공간을 HLS 색공간으로 변환 후 입력된 변화 적용
    c_hls = colorsys.rgb_to_hls(*c_rgb)
    h = c_hls[0] + dh
    l = max(min(c_hls[1] + dl, 1), 0)     # 0~1 범위를 넘지 않도록 제어
    s = max(min(c_hls[2] + ds, 1), 0)     # 0~1 범위를 넘지 않도록 제어
    
    # HLS 색공간에서 변경된 색을 RGB 색공간으로 변환
    c_rgb_new = colorsys.hls_to_rgb(h, l, s)
    alpha = max(min(alpha+dalpha, 1), 0)  # 0~1 범위를 넘지 않도록 제어
    
    return np.append(c_rgb_new, alpha) # alpha 추가하여 return


# #### 기존

# In[110]:


fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12,2), 
                        constrained_layout=True, 
                        sharex=True, sharey=True)

# basic stacked plot
sns.kdeplot(x='body_mass_g', data=df_peng, hue='sex', multiple='stack', 
            alpha=1, linewidths=0, ax=axs[0])

# 다양한 parameter 활용: palette, hue_order 등
sns.kdeplot(x='body_mass_g', data=df_peng, hue='sex', multiple='stack', 
            alpha=0.5, linewidths=0, ax=axs[1],
            palette=['darkred', 'C2'], hue_order=['Female', 'Male'])

plt.show()


# In[114]:


df_peng = sns.load_dataset('penguins')

fig, axs = plt.subplots(ncols=2, figsize=(10,4), constrained_layout=True, 
                        sharex=True, sharey=True)

sns.kdeplot(x='body_mass_g', data=df_peng, multiple='stack', 
            palette=['lightgrey', 'C0'], alpha=1, linewidths=0,
            hue='sex', ax=axs[0], hue_order=['Female', 'Male'])

sns.kdeplot(x='body_mass_g', data=df_peng, multiple='stack',
            palette=['lightgrey','C1'], alpha=1, linewidths=0,
            hue='sex', ax=axs[1])

# 성별 밀도 함수 객체 색상 변경
sexs = [axs[0].collections[0], axs[1].collections[0]]
for p in sexs:
    fc = p.get_facecolor()
    p.set_facecolor(set_hls(fc, dh=0.8, ds=-0.3, dalpha=0.2))

# handles 변경을 legend에 반영
handles = [axs[0].collections[1], axs[0].collections[0], axs[1].collections[0]]
labels = ["all sexs", "males", "females"]
axs[1].legend(handles=handles, labels=labels, loc="upper right")

plt.show()


# #### 03. Visualization - Bar plot

# #### 유채색 vs. 무채색
#  - 색을 이루는 구성 요소 3가지 >> 명도(Lightness), 색상(Hue), 채도(Saturation) >> HLS(hls)
#  - 무채색: 3가지 중 명도만 있는 경우
#    - 채도가 0이 되어 색상이 드러나지 않으면 무채색이 됨
#  - 채도를 아주 살짝 남기면 색상만 약하게 전달 가능

# #### set_hls( ) 함수
#  - c: (array-like, str) color in RGB space
#  - dh: (float) change in Hue // default=0
#  - dl: (float) change in Lightness // default=0
#  - ds: (float) change in Saturation // default=0

# #### 예시 코드

# In[116]:


# data
df_peng = sns.load_dataset("penguins")

# figure + axes
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(8, 2), constrained_layout=True)

# draw countplot
sns.countplot(y="species", data=df_peng, ax=axs)


# #### step by step

# In[117]:


axs


# In[118]:


axs.patches   # 3 bars = 3 patches


# In[119]:


# patch: Rectangle(), width, height, angle 을 요소로 가지고 있음.
for idx, p in enumerate(axs.patches): 
    print(idx, '//', p)


# In[120]:


# axes.patches.get_fc(): facecolor 4개 요소를 리턴 >> dh, dl, ds, alpha
for idx, p in enumerate(axs.patches): 
    print('idx:\t\t', idx)
    print('p:\t\t', p)
    print('p.get_fc():\t', p.get_fc())
    print()


# #### function decomposition: step by step

# In[121]:


import matplotlib.pyplot as plt
import seaborn as sns
df_peng = sns.load_dataset("penguins")   # data
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(8, 2), constrained_layout=True)   # figure + axes
sns.countplot(y="species", data=df_peng, ax=axs)   # draw countplot
p1, p2, p3 = axs.patches[0], axs.patches[1], axs.patches[2]


# In[124]:


p1   # axes.patches 중 하나의 patch.(여기서는 0번째 patch) >> Rectangle 모양.


# In[126]:


p1.get_fc()   # axes.patches 중 p1 patch의 face-color를 리턴하는 메서드 >> (Red, Green, Blue, Alpha) 값을 리턴.


# In[128]:


c1 = p1.get_fc()
c1


# In[130]:


mcolors


# In[131]:


mcolors.to_rgba(c1)


# In[135]:


c_rgba = mcolors.to_rgba(c1)
print('** c_rgba:', c_rgba)
print('** type of c_rgba:', type(c_rgba))

print()

c_rgb = c_rgba[:3]
alpha = c_rgba[3]
print('** c_rgb:', c_rgb)
print('** alpha:', alpha)


# In[101]:


c_hls = colorsys.rgb_to_hls(*c_rgb)
c_hls


# In[58]:


c1 = p1.get_fc()
display(c1, type(c1))


# In[59]:


display(    mcolors,    type(mcolors)   )


# In[60]:


c_rgba = mcolors.to_rgba(c1)
display(   c_rgba,    type(c_rgba)   )


# In[61]:


for idx, p in enumerate(ax.patches):
    if idx == 1:
        fc = s


# In[ ]:


# 응용
for idx, p in enumerate(ax.patches): 
    if idx in ([0,2]):
        


# In[ ]:


for i, p in enumerate(ax.patches):
    if i < 2:
        fc = set_hls(p.get_fc(), dl=0.4, ds=-0.3)
    else:
        fc = set_hls(p.get_fc(), ds=0.2)
    p.set_fc(fc)
    
display(fig)


# In[ ]:


for p in ax.patches:
    print(p)


# In[ ]:


ax.patches[0]


# In[ ]:


arr = ax.patches[0].get_fc()


# In[ ]:


set_hls(arr)


# In[ ]:


c_rgba = mcolors.to_rgba(c)


# In[ ]:


import matplotlib.colors as mcolors
import colorsys

def set_hls(c, dh=0, dl=0, ds=0, dalpha=0):
    """
    c : (array -like, str) color in RGB space
    dh : (float) change in Hue
        default = 0
    dl : (float) change in Lightness
        default = 0
    ds : (float) change in Saturation
        default = 0
    """
    # 입력된 color를 RGBA numpy array로 변환
    c_rgba = mcolors.to_rgba(c)
    
    # RGB와 alpha 분리
    c_rgb = c_rgba[:3]
    alpha = c_rgba[3]
    
    # RGB 색공간을 HLS 색공간으로 변환 후 입력된 변화 적용
    c_hls = colorsys.rgb_to_hls(*c_rgb)
    h = c_hls[0] + dh
    l = max(min(c_hls[1] + dl, 1), 0)     # 0~1 범위를 넘지 않도록 제어
    s = max(min(c_hls[2] + ds, 1), 0)     # 0~1 범위를 넘지 않도록 제어
    
    # HLS 색공간에서 변경된 색을 RGB 색공간으로 변환
    c_rgb_new = colorsys.hls_to_rgb(h, l, s)
    alpha = max(min(alpha+dalpha, 1), 0)  # 0~1 범위를 넘지 않도록 제어
    
    return np.append(c_rgb_new, alpha) # alpha 추가하여 return


# #### matplotlib & ax object - [Basic plots]
#  - https://matplotlib.org/stable/plot_types/index.html

# In[85]:


fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(12,6),
                        constrained_layout=True,
                        sharex=False, sharey=False)

# axs[0,0] - line plot
x = np.linspace(0, 10, 100)
y = 4 + 2 * np.sin(2*x)
axs[0,0].plot(x, y, linewidth=2.0)
axs[0,0].set(xlim=(0, 8), xticks=np.arange(1, 8), ylim=(0, 8), yticks=np.arange(1, 8))

# axs[0,1] - scatter plot
x = 4 + np.random.normal(0, 2, 24)
y = 4 + np.random.normal(0, 2, len(x))
sizes = np.random.uniform(15, 80, len(x))
colors = np.random.uniform(15, 80, len(x))
axs[0,1].scatter(x, y, s=sizes, c=colors, vmin=0, vmax=100)
axs[0,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[1,0] - bar plot
x = 0.5 + np.arange(8)
y = np.random.uniform(2, 7, len(x))
axs[1,0].bar(x, y, width=1, edgecolor='white', linewidth=0.7)
axs[1,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[1,1] - stem plot
# x = 0.5 + np.arange(8)
# y = np.random.uniform(2, 7, len(x))
# axs[1,1].stem(x, y)
# axs[1,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[1,1] - stem plot
x = [1, 2, 3, 4]
y = [0.4, 0.6, 0.9, 0.5]
axs[1,1].stem(x, y)
axs[1,1].set(xlim=(0,5), xticks=np.arange(1,5), ylim=(0,1), yticks=np.arange(0,1,0.1))

# axs[2,0] - step plot
x = 0.5 + np.arange(8)
y = np.random.uniform(2, 7, len(x))
axs[2,0].step(x, y, linewidth=2.5)
axs[2,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[2,1] - fill_between plot
x =np.linspace(0, 8, 16)
y1 = 3 + 4*x/8 + np.random.uniform(0, 0.5, len(x))
y2 = 1 + 2*x/8 + np.random.uniform(0, 0.5, len(x))
axs[2,1].fill_between(x, y1, y2, alpha=0.5, linewidth=0)
axs[2,1].plot(x, (y1+y2)/2, linewidth=2)
axs[2,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[3,0] - stackplot
x = np.arange(0, 10, 2)
ay = [1, 1.25, 2, 2.75, 3]
by = [1, 1, 1, 1, 1]
cy = [2, 1, 2, 1, 2]
y = np.vstack([ay, by, cy])
axs[3,0].stackplot(x, y)
axs[3,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[3,1] - contour(x,y,z)
x, y = np.meshgrid(np.linspace(-3, 3, 256), np.linspace(-3, 3, 256))
z = (1 - x/2 + x**5 + y**3) * np.exp(-x**2 - y**2)
levels = np.linspace(np.min(z), np.max(z), 7)
axs[3,1].contour(x, y, z, levels=levels)

plt.show()


# #### matplotlib & ax object - [Statistics plots]
#  - https://matplotlib.org/stable/plot_types/index.html

# In[99]:


fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(12,8),
                        constrained_layout=True,
                        sharex=False, sharey=False)


# axs[0,0] - histogram
x = 4 + np.random.normal(0, 1.5, 200)
axs[0,0].hist(x, bins=8, linewidth=0.5, edgecolor='white')
axs[0,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,56), yticks=np.linspace(0,56,9))


# axs[0,1] - boxplot
d = np.random.normal((3,5,4), (1.25,1.00, 1.25), (100,3))
vp = axs[0,1].boxplot(d, positions=[2,4,6], widths=1.5, patch_artist=True,
                      showmeans=False, showfliers=False,
                      medianprops={'color':'white', 'linewidth':0.5},
                      boxprops={'facecolor':'C0', 'edgecolor':'white', 'linewidth':0.5},
                      whiskerprops={'color':'C1', 'linewidth':1.5},
                      capprops={'color':'C2', 'linewidth':1.5})
axs[0,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


# axs[1,0] - errorbar
x = [2, 4, 6]
y = [3.6, 5, 4.2]
yerr = [0.9, 1.2, 0.5]
axs[1,0].errorbar(x, y, yerr, fmt='o', linewidth=2, capsize=6)
axs[1,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


# axs[1,1] - violinplot
d = np.random.normal((3,5,4), (0.75,1.00,0.75), (200,3))
vp = axs[1,1].violinplot(d, [2,4,6], widths=2, showmeans=False, showmedians=False, showextrema=False)
# for body in vp['bodies']:
#     body.set_alpha(0.9)
# axs[1,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


# axs[2,0] - eventplot
x = [2, 4, 6]
d = np.random.gamma(4, size=(3, 50))
axs[2,0].eventplot(d, orientation='vertical', lineoffsets=x, linewidth=0.75)
axs[2,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


# axs[2,1] - 2d histogram
x = np.random.randn(500)
y = 1.2 * x + np.random.randn(500) / 3
plt.style.use('_mpl-gallery-nogrid')
axs[2,1].hist2d(x, y, bins=(np.arange(-3, 3, 0.1), np.arange(-3, 3, 0.1)))
axs[2,1].set(xlim=(-2, 2), ylim=(-3, 3))


# axs[3,0] - hexbin plot
x = np.random.randn(500)
y = 1.2 * x + np.random.randn(500)/3
axs[3,0].hexbin(x, y, gridsize=20)
axs[3,0].set(xlim=(-2,2), ylim=(-3,3))


# axs[3,1] - pie plot
x = [1, 2, 3, 4]
colors = plt.get_cmap('Blues')(np.linspace(0.2, 0.7, len(x)))
axs[3,1].pie(x, colors=colors, radius=3, center=(4,4), 
             wedgeprops={'linewidth':1, 'edgecolor':'white'},
             frame=True)
axs[3,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

plt.show()


# #### E.O.D

# In[ ]:




#!/usr/bin/env python
# coding: utf-8

# # Visualization

# ## 00. load package 

# In[36]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import seaborn as sns
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt7
import colorsys

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
sns.set_style('whitegrid')


# ## 00. load data

# #### available datasets

# In[37]:


pd.DataFrame(sns.get_dataset_names()).T  


# In[38]:


# penguine data: df_peng
df_peng = sns.load_dataset('penguins')
display(
    df_peng.shape,
    df_peng.head(1), 
    df_peng.tail(1)
)


# ## 01. figure & axes

# #### 데이터 확인

# In[39]:


print(df_peng.shape); df_peng.head(1)


# #### 도화지/축 - (1,3) 레이아웃

# In[40]:


fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(12,1),
                       constrained_layout=True,
                       sharex=False, sharey=False)

print(ax[0], ax[1], ax[2])


# #### 도화지/축 - (2,3) 레이아웃

# In[41]:


fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(12,2),
                       constrained_layout=True,
                       sharex=False, sharey=False)

print(ax[0,0], ax[0,1], ax[0,2], ax[1,0], ax[1,1], ax[1,2])


# #### 레이아웃에 그래프 그리기

# In[42]:


# (1,2)
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12,2),   # nrows, ncols, figsize 설정
                        constrained_layout=True,            # subplot들 간의 간격을 최적으로 자동 조정
                        sharex=False, sharey=True)         # x축공유, y축공유 설정

sns.kdeplot(x='bill_length_mm', data=df_peng, ax=axs[0])
sns.kdeplot(x='bill_depth_mm', data=df_peng, ax=axs[1])

plt.show()

print('** Axes:', ax[0], ax[1])


# In[43]:


# (1,3)
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12,2),   # nrows, ncols, figsize 설정
                        constrained_layout=True,            # subplot들 간의 간격을 최적으로 자동 조정
                        sharex=False, sharey=False)         # x축공유, y축공유 설정

sns.kdeplot(x='bill_length_mm', data=df_peng, ax=axs[0])
sns.kdeplot(x='bill_depth_mm', data=df_peng, ax=axs[1])
sns.kdeplot(x='flipper_length_mm', data=df_peng, ax=axs[2])

plt.show()

print('** Axes:', axs[0], axs[1], axs[2])


# In[44]:


# (1,4) - for loop 이용
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(12,2),
                       constrained_layout=True,
                       sharex=False, sharey=False)
for i, v in zip(df_peng.columns[2:6], range(len(df_peng.columns[2:6]))):
    sns.kdeplot(x=i, data=df_peng, ax=axs[v])


# #### 잠깐 step by step

# ##### # step.1) zip

# In[45]:


for i, v in zip([1,2,3],[4,5,6]):
    print('** i:', i, '/ v:', v, ' **')


# ##### # step.2) plotting

# In[46]:


# (2,2)
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12,4),   # nrows, ncols, figsize 설정
                        constrained_layout=True,            # subplot들 간의 간격을 최적으로 자동 조정
                        sharex=False, sharey=False)         # x축공유, y축공유 설정

sns.kdeplot(x='bill_length_mm', data=df_peng, ax=axs[0,0])
sns.kdeplot(x='bill_depth_mm', data=df_peng, ax=axs[0,1])
sns.kdeplot(x='flipper_length_mm', data=df_peng, ax=axs[1,0])
sns.kdeplot(x='body_mass_g', data=df_peng, ax=axs[1,1])

plt.show()

print('** Axes:', axs[0,0], axs[0,1], axs[1,0], axs[1,1])


# In[47]:


# (3,2)

fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(12,6),   # nrows, ncols, figsize 설정
                        constrained_layout=True,            # subplot들 간의 간격을 최적으로 자동 조정
                        sharex=False, sharey=False)         # x축공유, y축공유 설정

sns.kdeplot(x='bill_length_mm', data=df_peng, ax=axs[0,0])
sns.kdeplot(x='bill_depth_mm', data=df_peng, ax=axs[0,1])
sns.kdeplot(x='flipper_length_mm', data=df_peng, ax=axs[1,0])
sns.kdeplot(x='body_mass_g', data=df_peng, ax=axs[1,1])
sns.kdeplot(x='flipper_length_mm', data=df_peng, ax=axs[2,0])
sns.kdeplot(x='body_mass_g', data=df_peng, ax=axs[2,1])

plt.show()

print('** Axes:', axs[0,0], axs[0,1], axs[1,0], axs[1,1], axs[2,0], axs[2,1])


# In[48]:


# (3,2)

fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(12,6),   # nrows, ncols, figsize 설정
                        constrained_layout=True,            # subplot들 간의 간격을 최적으로 자동 조정
                        sharex=False, sharey=False)         # x축공유, y축공유 설정

sns.kdeplot(x='bill_length_mm', data=df_peng, ax=axs[0,0])
sns.kdeplot(x='bill_depth_mm', data=df_peng, ax=axs[0,1])
sns.kdeplot(x='flipper_length_mm', data=df_peng, ax=axs[1,0])
sns.kdeplot(x='body_mass_g', data=df_peng, ax=axs[1,1])
sns.kdeplot(x='flipper_length_mm', data=df_peng, ax=axs[2,0])
sns.kdeplot(x='body_mass_g', data=df_peng, ax=axs[2,1])

plt.show()

print('** Axes:', axs[0,0], axs[0,1], axs[1,0], axs[1,1], axs[2,0], axs[2,1])


# #### matplotlib & ax object - [Basic plots]
#  - https://matplotlib.org/stable/plot_types/index.html

# In[51]:


# 8 Basic Plots

fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(12,6),
                        constrained_layout=True,
                        sharex=False, sharey=False)

# axs[0,0] - line plot
x = np.linspace(0, 10, 100)
y = 4 + 2 * np.sin(2*x)
axs[0,0].plot(x, y, linewidth=2.0)
axs[0,0].set(xlim=(0, 8), xticks=np.arange(1, 8), ylim=(0, 8), yticks=np.arange(1, 8))

# axs[0,1] - scatter plot
x = 4 + np.random.normal(0, 2, 24)
y = 4 + np.random.normal(0, 2, len(x))
sizes = np.random.uniform(15, 80, len(x))
colors = np.random.uniform(15, 80, len(x))
axs[0,1].scatter(x, y, s=sizes, c=colors, vmin=0, vmax=100)
axs[0,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[1,0] - bar plot
x = 0.5 + np.arange(8)
y = np.random.uniform(2, 7, len(x))
axs[1,0].bar(x, y, width=1, edgecolor='white', linewidth=0.7)
axs[1,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[1,1] - stem plot
x = [1, 2, 3, 4]
y = [0.4, 0.6, 0.9, 0.5]
axs[1,1].stem(x, y)
axs[1,1].set(xlim=(0,5), xticks=np.arange(1,5), ylim=(0,1), yticks=np.arange(0,1,0.1))

# axs[2,0] - step plot
x = 0.5 + np.arange(8)
y = np.random.uniform(2, 7, len(x))
axs[2,0].step(x, y, linewidth=2.5)
axs[2,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[2,1] - fill_between plot
x =np.linspace(0, 8, 16)
y1 = 3 + 4*x/8 + np.random.uniform(0, 0.5, len(x))
y2 = 1 + 2*x/8 + np.random.uniform(0, 0.5, len(x))
axs[2,1].fill_between(x, y1, y2, alpha=0.5, linewidth=0)
axs[2,1].plot(x, (y1+y2)/2, linewidth=2)
axs[2,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[3,0] - stackplot
x = np.arange(0, 10, 2)
ay = [1, 1.25, 2, 2.75, 3]
by = [1, 1, 1, 1, 1]
cy = [2, 1, 2, 1, 2]
y = np.vstack([ay, by, cy])
axs[3,0].stackplot(x, y)
axs[3,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[3,1] - contour(x,y,z)
x, y = np.meshgrid(np.linspace(-3, 3, 256), np.linspace(-3, 3, 256))
z = (1 - x/2 + x**5 + y**3) * np.exp(-x**2 - y**2)
levels = np.linspace(np.min(z), np.max(z), 7)
axs[3,1].contour(x, y, z, levels=levels)

plt.show()


# #### matplotlib & ax object - [Statistics plots]
#  - https://matplotlib.org/stable/plot_types/index.html

# In[52]:


# 8 Statistics Plots

fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(12,8),
                        constrained_layout=True,
                        sharex=False, sharey=False)


# axs[0,0] - histogram
x = 4 + np.random.normal(0, 1.5, 200)
axs[0,0].hist(x, bins=8, linewidth=0.5, edgecolor='white')
axs[0,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,56), yticks=np.linspace(0,56,9))


# axs[0,1] - boxplot
d = np.random.normal((3,5,4), (1.25,1.00, 1.25), (100,3))
vp = axs[0,1].boxplot(d, positions=[2,4,6], widths=1.5, patch_artist=True,
                      showmeans=False, showfliers=False,
                      medianprops={'color':'white', 'linewidth':0.5},
                      boxprops={'facecolor':'C0', 'edgecolor':'white', 'linewidth':0.5},
                      whiskerprops={'color':'C1', 'linewidth':1.5},
                      capprops={'color':'C2', 'linewidth':1.5})
axs[0,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


# axs[1,0] - errorbar
x = [2, 4, 6]
y = [3.6, 5, 4.2]
yerr = [0.9, 1.2, 0.5]
axs[1,0].errorbar(x, y, yerr, fmt='o', linewidth=2, capsize=6)
axs[1,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


# axs[1,1] - violinplot
d = np.random.normal((3,5,4), (0.75,1.00,0.75), (200,3))
vp = axs[1,1].violinplot(d, [2,4,6], widths=2, showmeans=False, showmedians=False, showextrema=False)
# for body in vp['bodies']:
#     body.set_alpha(0.9)
# axs[1,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


# axs[2,0] - eventplot
x = [2, 4, 6]
d = np.random.gamma(4, size=(3, 50))
axs[2,0].eventplot(d, orientation='vertical', lineoffsets=x, linewidth=0.75)
axs[2,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


# axs[2,1] - 2d histogram
x = np.random.randn(500)
y = 1.2 * x + np.random.randn(500) / 3
plt.style.use('_mpl-gallery-nogrid')
axs[2,1].hist2d(x, y, bins=(np.arange(-3, 3, 0.1), np.arange(-3, 3, 0.1)))
axs[2,1].set(xlim=(-2, 2), ylim=(-3, 3))


# axs[3,0] - hexbin plot
x = np.random.randn(500)
y = 1.2 * x + np.random.randn(500)/3
axs[3,0].hexbin(x, y, gridsize=20)
axs[3,0].set(xlim=(-2,2), ylim=(-3,3))


# axs[3,1] - pie plot
x = [1, 2, 3, 4]
colors = plt.get_cmap('Blues')(np.linspace(0.2, 0.7, len(x)))
axs[3,1].pie(x, colors=colors, radius=3, center=(4,4), 
             wedgeprops={'linewidth':1, 'edgecolor':'white'},
             frame=True)
axs[3,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

plt.show()


# #### E.O.D










#!/usr/bin/env python
# coding: utf-8

# # Visualization

# ## 00. load package 

# In[68]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import seaborn as sns
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt7
import colorsys

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
sns.set_style('whitegrid')


# ## 00. load data

# #### available datasets

# In[69]:


pd.DataFrame(sns.get_dataset_names()).T  


# In[70]:


# titanic data: df_titanic
df_titanic = sns.load_dataset('titanic')
display(
    df_titanic.shape,
    df_titanic.head(1), 
    df_titanic.tail(1)
)


# In[71]:


# penguine data: df_peng
df_peng = sns.load_dataset('penguins')
display(
    df_peng.shape,
    df_peng.head(1), 
    df_peng.tail(1)
)


# In[73]:


# tip data: df_tip
df_tips = sns.load_dataset('tips')
display(
    df_tips.shape,
    df_tips.head(1), 
    df_tips.tail(1)
)


# ## 01. figure & axes 시각화 Template

# #### 데이터 확인

# In[15]:


print(df_peng.shape); df_peng.head(1)


# #### Figure & Axes

# In[16]:


# (3,2)
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(12,6),   # nrows, ncols, figsize 설정
                        constrained_layout=True,            # subplot들 간의 간격을 최적으로 자동 조정
                        sharex=False, sharey=False)         # x축공유, y축공유 설정

sns.kdeplot(x='bill_length_mm', data=df_peng, ax=axs[0,0])
sns.kdeplot(x='bill_depth_mm', data=df_peng, ax=axs[0,1])
sns.kdeplot(x='flipper_length_mm', data=df_peng, ax=axs[1,0])
sns.kdeplot(x='body_mass_g', data=df_peng, ax=axs[1,1])
sns.kdeplot(x='flipper_length_mm', data=df_peng, ax=axs[2,0])
sns.kdeplot(x='body_mass_g', data=df_peng, ax=axs[2,1])

print('** Axes:', axs[0,0], axs[0,1], axs[1,0], axs[1,1], axs[2,0], axs[2,1])


# ## 02. Visualization - Matplotlib

# #### matplotlib & ax object - [Basic plots]
#  - line plot, scatter plot, bar plot, stem plot, step plot, fill_between plot, stackplot, contour plot
#  - https://matplotlib.org/stable/plot_types/index.html

# In[17]:


fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(12,6),
                        constrained_layout=True,
                        sharex=False, sharey=False)

# axs[0,0] - line plot
x = np.linspace(0, 10, 100)
y = 4 + 2 * np.sin(2*x)
axs[0,0].plot(x, y, linewidth=2.0)
axs[0,0].set(xlim=(0, 8), xticks=np.arange(1, 8), ylim=(0, 8), yticks=np.arange(1, 8))

# axs[0,1] - scatter plot
x = 4 + np.random.normal(0, 2, 24)
y = 4 + np.random.normal(0, 2, len(x))
sizes = np.random.uniform(15, 80, len(x))
colors = np.random.uniform(15, 80, len(x))
axs[0,1].scatter(x, y, s=sizes, c=colors, vmin=0, vmax=100)
axs[0,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[1,0] - bar plot
x = 0.5 + np.arange(8)
y = np.random.uniform(2, 7, len(x))
axs[1,0].bar(x, y, width=1, edgecolor='white', linewidth=0.7)
axs[1,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[1,1] - stem plot
# x = 0.5 + np.arange(8)
# y = np.random.uniform(2, 7, len(x))
# axs[1,1].stem(x, y)
# axs[1,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[1,1] - stem plot
x = [1, 2, 3, 4]
y = [0.4, 0.6, 0.9, 0.5]
axs[1,1].stem(x, y)
axs[1,1].set(xlim=(0,5), xticks=np.arange(1,5), ylim=(0,1), yticks=np.arange(0,1,0.1))

# axs[2,0] - step plot
x = 0.5 + np.arange(8)
y = np.random.uniform(2, 7, len(x))
axs[2,0].step(x, y, linewidth=2.5)
axs[2,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[2,1] - fill_between plot
x =np.linspace(0, 8, 16)
y1 = 3 + 4*x/8 + np.random.uniform(0, 0.5, len(x))
y2 = 1 + 2*x/8 + np.random.uniform(0, 0.5, len(x))
axs[2,1].fill_between(x, y1, y2, alpha=0.5, linewidth=0)
axs[2,1].plot(x, (y1+y2)/2, linewidth=2)
axs[2,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[3,0] - stackplot
x = np.arange(0, 10, 2)
ay = [1, 1.25, 2, 2.75, 3]
by = [1, 1, 1, 1, 1]
cy = [2, 1, 2, 1, 2]
y = np.vstack([ay, by, cy])
axs[3,0].stackplot(x, y)
axs[3,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[3,1] - contour(x,y,z)
x, y = np.meshgrid(np.linspace(-3, 3, 256), np.linspace(-3, 3, 256))
z = (1 - x/2 + x**5 + y**3) * np.exp(-x**2 - y**2)
levels = np.linspace(np.min(z), np.max(z), 7)
axs[3,1].contour(x, y, z, levels=levels)

plt.show()


# #### matplotlib & ax object - [Statistics plots]
#  - histogram, boxplot, errorbar, violinplot, eventplot, 2d histogram, hexbin plot, pie plot
#  - https://matplotlib.org/stable/plot_types/index.html

# In[18]:


fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(12,8),
                        constrained_layout=True,
                        sharex=False, sharey=False)


# axs[0,0] - histogram
x = 4 + np.random.normal(0, 1.5, 200)
axs[0,0].hist(x, bins=8, linewidth=0.5, edgecolor='white')
axs[0,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,56), yticks=np.linspace(0,56,9))


# axs[0,1] - boxplot
d = np.random.normal((3,5,4), (1.25,1.00, 1.25), (100,3))
vp = axs[0,1].boxplot(d, positions=[2,4,6], widths=1.5, patch_artist=True,
                      showmeans=False, showfliers=False,
                      medianprops={'color':'white', 'linewidth':0.5},
                      boxprops={'facecolor':'C0', 'edgecolor':'white', 'linewidth':0.5},
                      whiskerprops={'color':'C1', 'linewidth':1.5},
                      capprops={'color':'C2', 'linewidth':1.5})
axs[0,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


# axs[1,0] - errorbar
x = [2, 4, 6]
y = [3.6, 5, 4.2]
yerr = [0.9, 1.2, 0.5]
axs[1,0].errorbar(x, y, yerr, fmt='o', linewidth=2, capsize=6)
axs[1,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


# axs[1,1] - violinplot
d = np.random.normal((3,5,4), (0.75,1.00,0.75), (200,3))
vp = axs[1,1].violinplot(d, [2,4,6], widths=2, showmeans=False, showmedians=False, showextrema=False)
# for body in vp['bodies']:
#     body.set_alpha(0.9)
# axs[1,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


# axs[2,0] - eventplot
x = [2, 4, 6]
d = np.random.gamma(4, size=(3, 50))
axs[2,0].eventplot(d, orientation='vertical', lineoffsets=x, linewidth=0.75)
axs[2,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


# axs[2,1] - 2d histogram
x = np.random.randn(500)
y = 1.2 * x + np.random.randn(500) / 3
plt.style.use('_mpl-gallery-nogrid')
axs[2,1].hist2d(x, y, bins=(np.arange(-3, 3, 0.1), np.arange(-3, 3, 0.1)))
axs[2,1].set(xlim=(-2, 2), ylim=(-3, 3))


# axs[3,0] - hexbin plot
x = np.random.randn(500)
y = 1.2 * x + np.random.randn(500)/3
axs[3,0].hexbin(x, y, gridsize=20)
axs[3,0].set(xlim=(-2,2), ylim=(-3,3))


# axs[3,1] - pie plot
x = [1, 2, 3, 4]
colors = plt.get_cmap('Blues')(np.linspace(0.2, 0.7, len(x)))
axs[3,1].pie(x, colors=colors, radius=3, center=(4,4), 
             wedgeprops={'linewidth':1, 'edgecolor':'white'},
             frame=True)
axs[3,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


plt.show()


# ## 03. Visualization - Seaborn

# #### seaborn - Distribution of Data
#  - 데이터: df_titanic, df_peng

# In[33]:


sns.displot(df_peng, x='flipper_length_mm')   # 데이터프레임에서 컬럼 하나의 분포 보기


# In[20]:


sns.displot(df_peng, x='flipper_length_mm', binwidth=3)   # 히스토그램 막대의 bin 크기 지정하기


# In[24]:


sns.displot(df_peng, x='flipper_length_mm', bins=20)   # 히스토그램의 막대 개수 지정하기


# In[75]:


display(df_tips.shape, df_tips.head(2))
sns.displot(df_tips, x='size')


# In[76]:


display(pd.DataFrame(df_tips['size'].value_counts()).T)   # show values

sns.displot(df_tips, x='size', bins=[1,2,3,4,5,6,7])   # plotting


# In[77]:


# discrete=True
display(pd.DataFrame(df_tips['size'].value_counts()).T)   # show values
sns.displot(df_tips, x='size', discrete=True)   # plotting


# In[79]:


# shrink
display(pd.DataFrame(df_tips['size'].value_counts()).T)   # show values
sns.displot(df_tips, x='size', shrink=.8)   # plotting


# In[80]:


# shrink
display(pd.DataFrame(df_tips['day'].value_counts()).T)   # show values
sns.displot(df_tips, x='day', shrink=.8)   # plotting


# In[ ]:





# In[ ]:





# In[ ]:





# In[ ]:





# In[ ]:





# In[ ]:





# #### set figure size @ seaborn(sns)

# In[81]:


sns.displot(df_peng, x='flipper_length_mm')


# In[50]:


# Method.1) Change the size of Axes-Level Plots
fig, axes = plt.subplots(figsize=(12,2))
# sns.scatterplot(data=df_peng, x='flipper_length_mm', ax=axes)
# display scatterplot
sns.scatterplot(data=df, x="var1", y="var2")


# In[38]:


import pandas as pd
import seaborn as sns

#create data
df = pd.DataFrame({"var1": [25, 12, 15, 14, 19, 23, 25, 29],
                   "var2": [5, 7, 7, 9, 12, 9, 9, 4],
                   "var3": [11, 8, 10, 6, 6, 5, 9, 12]})

#define figure size
sns.set(rc={"figure.figsize":(8, 4)}) #width=8, height=4

#display scatterplot
sns.scatterplot(data=df, x="var1", y="var2")


# In[67]:


# Method.2) 
sns.set(rc={"figure.figsize":(12, 2)})  
sns.displot(df_peng, x='flipper_length_mm')  


# In[ ]:





# In[ ]:





# #### Reference
# * seaborn 시각화 >> https://seaborn.pydata.org/tutorial/distributions.html
# * 그래프 시각화/main 강조 >> https://jehyunlee.github.io/2022/09/05/Python-DS-112-sethls/
# * matplotlib 로컬 파일 위치 >> file:///C:/Users/donghwank/Downloads/matplotlib.html
# * matplotlib >> https://steadiness-193.tistory.com/m/174
# * VISUALIZATION.html >> file:///C:/Users/donghwank/Downloads/VISUALIZATION.html

# In[ ]:








#!/usr/bin/env python
# coding: utf-8

# # Visualization

# ## 00. load package 

# In[1]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import seaborn as sns
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt7
import colorsys

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
sns.set_style('whitegrid')


# ## 00. load data

# In[2]:


pd.DataFrame(sns.get_dataset_names()).T   # available datasets


# In[3]:


# penguine data: df_peng
df_peng = sns.load_dataset('penguins')
display(df_peng.shape, df_peng.head(1), df_peng.tail(1))


# #### UDF: set_hls( )
#  - 기능: RGB 컬러 값을 전달받아 hls 값을 원하는 값으로 설정
#  - h: dh >> change in hue (float)
#  - l: dl >> change in Lightness (float)
#  - s: ds >> change in Saturation (float)

# In[4]:


def set_hls(c, dh=0, dl=0, ds=0, dalpha=0):
    """
    c : (array -like, str) color in RGB space
    dh : (float) change in Hue
         default = 0
    dl : (float) change in Lightness
         default = 0
    ds : (float) change in Saturation
         default = 0
    dalpha : opacity(투명도)
         default = 0
         >> 보통 alpha 값이 1이면 완전 불투명, 0이면 완전 투명
    """

    # 입력된 color를 RGBA numpy array로 변환
    c_rgba = mcolors.to_rgba(c)
    
    # RGB와 alpha 분리
    c_rgb = c_rgba[:3]
    alpha = c_rgba[3]
    
    # RGB 색공간을 HLS 색공간으로 변환 후 입력된 변화 적용
    c_hls = colorsys.rgb_to_hls(*c_rgb)
    h = c_hls[0] + dh
    l = max(min(c_hls[1] + dl, 1), 0)     # 0~1 범위를 넘지 않도록 제어
    s = max(min(c_hls[2] + ds, 1), 0)     # 0~1 범위를 넘지 않도록 제어
    
    # HLS 색공간에서 변경된 색을 RGB 색공간으로 변환
    c_rgb_new = colorsys.hls_to_rgb(h, l, s)
    alpha = max(min(alpha+dalpha, 1), 0)  # 0~1 범위를 넘지 않도록 제어
    
    return np.append(c_rgb_new, alpha) # alpha 추가하여 return


# #### 03. Visualization - Bar plot

# #### 유채색 vs. 무채색
#  - 색을 이루는 구성 요소 3가지 >> 명도(Lightness), 색상(Hue), 채도(Saturation) >> HLS(hls)
#  - 무채색: 3가지 중 명도만 있는 경우
#    - 채도가 0이 되어 색상이 드러나지 않으면 무채색이 됨
#  - 채도를 아주 살짝 남기면 색상만 약하게 전달 가능

# #### set_hls( ) 함수
#  - c: (array-like, str) color in RGB space
#  - dh: (float) change in Hue // default=0
#  - dl: (float) change in Lightness // default=0
#  - ds: (float) change in Saturation // default=0

# #### 예시 코드

# In[116]:


# data
df_peng = sns.load_dataset("penguins")

# figure + axes
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(8, 2), constrained_layout=True)

# draw countplot
sns.countplot(y="species", data=df_peng, ax=axs)


# #### step by step

# In[117]:


axs


# In[118]:


axs.patches   # 3 bars = 3 patches


# In[119]:


# patch: Rectangle(), width, height, angle 을 요소로 가지고 있음.
for idx, p in enumerate(axs.patches): 
    print(idx, '//', p)


# In[120]:


# axes.patches.get_fc(): facecolor 4개 요소를 리턴 >> dh, dl, ds, alpha
for idx, p in enumerate(axs.patches): 
    print('idx:\t\t', idx)
    print('p:\t\t', p)
    print('p.get_fc():\t', p.get_fc())
    print()


# #### function decomposition: step by step

# In[121]:


import matplotlib.pyplot as plt
import seaborn as sns
df_peng = sns.load_dataset("penguins")   # data
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(8, 2), constrained_layout=True)   # figure + axes
sns.countplot(y="species", data=df_peng, ax=axs)   # draw countplot
p1, p2, p3 = axs.patches[0], axs.patches[1], axs.patches[2]


# In[124]:


p1   # axes.patches 중 하나의 patch.(여기서는 0번째 patch) >> Rectangle 모양.


# In[126]:


p1.get_fc()   # axes.patches 중 p1 patch의 face-color를 리턴하는 메서드 >> (Red, Green, Blue, Alpha) 값을 리턴.


# In[128]:


c1 = p1.get_fc()
c1


# In[130]:


mcolors


# In[131]:


mcolors.to_rgba(c1)


# In[135]:


c_rgba = mcolors.to_rgba(c1)
print('** c_rgba:', c_rgba)
print('** type of c_rgba:', type(c_rgba))

print()

c_rgb = c_rgba[:3]
alpha = c_rgba[3]
print('** c_rgb:', c_rgb)
print('** alpha:', alpha)


# In[101]:


c_hls = colorsys.rgb_to_hls(*c_rgb)
c_hls


# In[58]:


c1 = p1.get_fc()
display(c1, type(c1))


# In[59]:


display(    mcolors,    type(mcolors)   )


# In[60]:


c_rgba = mcolors.to_rgba(c1)
display(   c_rgba,    type(c_rgba)   )


# In[61]:


for idx, p in enumerate(ax.patches):
    if idx == 1:
        fc = s


# In[ ]:


# 응용
for idx, p in enumerate(ax.patches): 
    if idx in ([0,2]):
        


# In[ ]:


for i, p in enumerate(ax.patches):
    if i < 2:
        fc = set_hls(p.get_fc(), dl=0.4, ds=-0.3)
    else:
        fc = set_hls(p.get_fc(), ds=0.2)
    p.set_fc(fc)
    
display(fig)


# In[ ]:


for p in ax.patches:
    print(p)


# In[ ]:


ax.patches[0]


# In[ ]:


arr = ax.patches[0].get_fc()


# In[ ]:


set_hls(arr)


# In[ ]:


c_rgba = mcolors.to_rgba(c)


# In[ ]:


import matplotlib.colors as mcolors
import colorsys

def set_hls(c, dh=0, dl=0, ds=0, dalpha=0):
    """
    c : (array -like, str) color in RGB space
    dh : (float) change in Hue
        default = 0
    dl : (float) change in Lightness
        default = 0
    ds : (float) change in Saturation
        default = 0
    """
    # 입력된 color를 RGBA numpy array로 변환
    c_rgba = mcolors.to_rgba(c)
    
    # RGB와 alpha 분리
    c_rgb = c_rgba[:3]
    alpha = c_rgba[3]
    
    # RGB 색공간을 HLS 색공간으로 변환 후 입력된 변화 적용
    c_hls = colorsys.rgb_to_hls(*c_rgb)
    h = c_hls[0] + dh
    l = max(min(c_hls[1] + dl, 1), 0)     # 0~1 범위를 넘지 않도록 제어
    s = max(min(c_hls[2] + ds, 1), 0)     # 0~1 범위를 넘지 않도록 제어
    
    # HLS 색공간에서 변경된 색을 RGB 색공간으로 변환
    c_rgb_new = colorsys.hls_to_rgb(h, l, s)
    alpha = max(min(alpha+dalpha, 1), 0)  # 0~1 범위를 넘지 않도록 제어
    
    return np.append(c_rgb_new, alpha) # alpha 추가하여 return


# #### matplotlib & ax object - [Basic plots]
#  - https://matplotlib.org/stable/plot_types/index.html

# In[85]:


fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(12,6),
                        constrained_layout=True,
                        sharex=False, sharey=False)

# axs[0,0] - line plot
x = np.linspace(0, 10, 100)
y = 4 + 2 * np.sin(2*x)
axs[0,0].plot(x, y, linewidth=2.0)
axs[0,0].set(xlim=(0, 8), xticks=np.arange(1, 8), ylim=(0, 8), yticks=np.arange(1, 8))

# axs[0,1] - scatter plot
x = 4 + np.random.normal(0, 2, 24)
y = 4 + np.random.normal(0, 2, len(x))
sizes = np.random.uniform(15, 80, len(x))
colors = np.random.uniform(15, 80, len(x))
axs[0,1].scatter(x, y, s=sizes, c=colors, vmin=0, vmax=100)
axs[0,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[1,0] - bar plot
x = 0.5 + np.arange(8)
y = np.random.uniform(2, 7, len(x))
axs[1,0].bar(x, y, width=1, edgecolor='white', linewidth=0.7)
axs[1,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[1,1] - stem plot
# x = 0.5 + np.arange(8)
# y = np.random.uniform(2, 7, len(x))
# axs[1,1].stem(x, y)
# axs[1,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[1,1] - stem plot
x = [1, 2, 3, 4]
y = [0.4, 0.6, 0.9, 0.5]
axs[1,1].stem(x, y)
axs[1,1].set(xlim=(0,5), xticks=np.arange(1,5), ylim=(0,1), yticks=np.arange(0,1,0.1))

# axs[2,0] - step plot
x = 0.5 + np.arange(8)
y = np.random.uniform(2, 7, len(x))
axs[2,0].step(x, y, linewidth=2.5)
axs[2,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[2,1] - fill_between plot
x =np.linspace(0, 8, 16)
y1 = 3 + 4*x/8 + np.random.uniform(0, 0.5, len(x))
y2 = 1 + 2*x/8 + np.random.uniform(0, 0.5, len(x))
axs[2,1].fill_between(x, y1, y2, alpha=0.5, linewidth=0)
axs[2,1].plot(x, (y1+y2)/2, linewidth=2)
axs[2,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[3,0] - stackplot
x = np.arange(0, 10, 2)
ay = [1, 1.25, 2, 2.75, 3]
by = [1, 1, 1, 1, 1]
cy = [2, 1, 2, 1, 2]
y = np.vstack([ay, by, cy])
axs[3,0].stackplot(x, y)
axs[3,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

# axs[3,1] - contour(x,y,z)
x, y = np.meshgrid(np.linspace(-3, 3, 256), np.linspace(-3, 3, 256))
z = (1 - x/2 + x**5 + y**3) * np.exp(-x**2 - y**2)
levels = np.linspace(np.min(z), np.max(z), 7)
axs[3,1].contour(x, y, z, levels=levels)

plt.show()


# #### matplotlib & ax object - [Statistics plots]
#  - https://matplotlib.org/stable/plot_types/index.html

# In[99]:


fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(12,8),
                        constrained_layout=True,
                        sharex=False, sharey=False)


# axs[0,0] - histogram
x = 4 + np.random.normal(0, 1.5, 200)
axs[0,0].hist(x, bins=8, linewidth=0.5, edgecolor='white')
axs[0,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,56), yticks=np.linspace(0,56,9))


# axs[0,1] - boxplot
d = np.random.normal((3,5,4), (1.25,1.00, 1.25), (100,3))
vp = axs[0,1].boxplot(d, positions=[2,4,6], widths=1.5, patch_artist=True,
                      showmeans=False, showfliers=False,
                      medianprops={'color':'white', 'linewidth':0.5},
                      boxprops={'facecolor':'C0', 'edgecolor':'white', 'linewidth':0.5},
                      whiskerprops={'color':'C1', 'linewidth':1.5},
                      capprops={'color':'C2', 'linewidth':1.5})
axs[0,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


# axs[1,0] - errorbar
x = [2, 4, 6]
y = [3.6, 5, 4.2]
yerr = [0.9, 1.2, 0.5]
axs[1,0].errorbar(x, y, yerr, fmt='o', linewidth=2, capsize=6)
axs[1,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


# axs[1,1] - violinplot
d = np.random.normal((3,5,4), (0.75,1.00,0.75), (200,3))
vp = axs[1,1].violinplot(d, [2,4,6], widths=2, showmeans=False, showmedians=False, showextrema=False)
# for body in vp['bodies']:
#     body.set_alpha(0.9)
# axs[1,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


# axs[2,0] - eventplot
x = [2, 4, 6]
d = np.random.gamma(4, size=(3, 50))
axs[2,0].eventplot(d, orientation='vertical', lineoffsets=x, linewidth=0.75)
axs[2,0].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))


# axs[2,1] - 2d histogram
x = np.random.randn(500)
y = 1.2 * x + np.random.randn(500) / 3
plt.style.use('_mpl-gallery-nogrid')
axs[2,1].hist2d(x, y, bins=(np.arange(-3, 3, 0.1), np.arange(-3, 3, 0.1)))
axs[2,1].set(xlim=(-2, 2), ylim=(-3, 3))


# axs[3,0] - hexbin plot
x = np.random.randn(500)
y = 1.2 * x + np.random.randn(500)/3
axs[3,0].hexbin(x, y, gridsize=20)
axs[3,0].set(xlim=(-2,2), ylim=(-3,3))


# axs[3,1] - pie plot
x = [1, 2, 3, 4]
colors = plt.get_cmap('Blues')(np.linspace(0.2, 0.7, len(x)))
axs[3,1].pie(x, colors=colors, radius=3, center=(4,4), 
             wedgeprops={'linewidth':1, 'edgecolor':'white'},
             frame=True)
axs[3,1].set(xlim=(0,8), xticks=np.arange(1,8), ylim=(0,8), yticks=np.arange(1,8))

plt.show()


# #### E.O.D

# In[ ]:



